In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import ast
from collections import namedtuple

Case = namedtuple('Case', 'expr mode')

cases = [
    Case('2 + 3*4 + x', 'eval'),
    Case('for i in range(10): print(i)', 'exec')
]

for case in cases:
    ex = ast.parse(case.expr, mode=case.mode)
    print(ast.dump(ex))
    print('\n' + '=' * 80 + '\n')

Expression(body=BinOp(left=BinOp(left=Constant(value=2), op=Add(), right=BinOp(left=Constant(value=3), op=Mult(), right=Constant(value=4))), op=Add(), right=Name(id='x', ctx=Load())))


Module(body=[For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Constant(value=10)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='i', ctx=Load())], keywords=[]))], orelse=[])], type_ignores=[])




In [4]:
import ast


class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.loaded = set()
        self.stored = set()
        self.deleted = set()

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.loaded.add(node.id)
        elif isinstance(node.ctx, ast.Store):
            self.stored.add(node.id)
        elif isinstance(node.ctx, ast.Del):
            self.deleted.add(node.id)


expr = '''
for i in range(10):
    print(i)
del i
'''
root = ast.parse(expr, mode='exec')
analyzer = CodeAnalyzer()
analyzer.visit(root)

print('loaded:', analyzer.loaded)
print('stored:', analyzer.stored)
print('deleted:', analyzer.deleted)

exec(compile(root, '<stdin>', 'exec'))

loaded: {'i', 'range', 'print'}
stored: {'i'}
deleted: {'i'}
0
1
2
3
4
5
6
7
8
9


In [5]:
import ast
import inspect


class NameLower(ast.NodeVisitor):
    def __init__(self, lowered_names):
        self.lowered_names = lowered_names

    def visit_FunctionDef(self, node):
        code = '__globals = globals()\n'
        code += '\n'.join("{0} = __globals['{0}']".format(name)
                          for name in self.lowered_names)
        code_ast = ast.parse(code, mode='exec')
        node.body[:0] = code_ast.body
        self.func = node


def lower_names(*namelist):
    def lower(func):
        srclines = inspect.getsource(func).splitlines()
        for n, line in enumerate(srclines):
            if '@lower_names' in line:
                break

        src = '\n'.join(srclines[n+1:])
        if src.startswith((' ', '\t')):
            src = 'if 1:\n' + src

        top = ast.parse(src, mode='exec')
        cl = NameLower(namelist)
        cl.visit(top)

        temp = {}
        exec(compile(top, '', 'exec'), temp, temp)

        func.__code__ = temp[func.__name__].__code__
        return func

    return lower


INCR = 1


@lower_names('INCR')
def countdown(n):
    while n > 0:
        n -= INCR


countdown(1e+7)